# CEQR API Test Notebook

This notebook tests the CEQR (City Environmental Quality Review) API call using Python requests.

The API endpoint is: `https://a002-ceqraccess.nyc.gov/ceqr/`

This is a POST request with form data that searches for CEQR projects by block and lot.


In [1]:
import requests
from urllib.parse import unquote
import json

print("✅ Imports successful")


✅ Imports successful


## API Request Configuration

Setting up the URL, headers, and body from the fetch call.


In [ ]:
# API endpoint
url = "https://a002-ceqraccess.nyc.gov/ceqr/"

# Minimal headers - only what's essential
headers = {
    "content-type": "application/x-www-form-urlencoded",
    "referer": "https://a002-ceqraccess.nyc.gov/ceqr/"
}

# Minimal body - ONLY borough, block, and lot (no VIEWSTATE, no other fields)
from urllib.parse import urlencode

form_data = {
    "ctl00$MainContent$ddlBorough": "Brooklyn",
    "ctl00$MainContent$txtBlock": "7061",
    "ctl00$MainContent$txtLot": "27"
}

body = urlencode(form_data)

print(f"URL: {url}")
print(f"Headers: {headers}")
print(f"Body: {body}")
print(f"\n✅ Minimal request configured with only borough, block, and lot")


URL: https://a002-ceqraccess.nyc.gov/ceqr/
Headers configured: 13 headers
Body length: 17893 characters


## Make the POST Request

Sending the POST request with the configured headers and body.


In [ ]:
# Make the POST request with minimal data
try:
    response = requests.post(url, headers=headers, data=body, timeout=30)
    
    print(f"Status Code: {response.status_code}")
    print(f"Response Content Length: {len(response.content)} bytes")
    
    # Check if request was successful
    if response.status_code == 200:
        print("\n✅ Request successful!")
        
        # Check for error pages
        if 'Error' in response.text or 'Unhandled' in response.text:
            print("⚠️  Response contains error page")
        elif 'grdSearchResults' in response.text or 'Search Results' in response.text:
            print("✅ Found search results in response!")
        else:
            print("📄 Response received (checking content...)")
        
        # Show first 500 characters to see what we got
        print("\n--- First 500 characters of response ---")
        print(response.text[:500])
        
        # Check if we can find any CEQR numbers or project info
        import re
        ceqr_pattern = r'\d{2}[A-Z]{3}\d{3}[A-Z]'
        ceqr_matches = re.findall(ceqr_pattern, response.text)
        if ceqr_matches:
            print(f"\n✅ Found CEQR numbers: {set(ceqr_matches)}")
        
    else:
        print(f"\n❌ Request failed with status code: {response.status_code}")
        print(f"Response: {response.text[:500]}")
        
except requests.exceptions.RequestException as e:
    print(f"❌ Error making request: {e}")
except Exception as e:
    print(f"❌ Unexpected error: {e}")


Status Code: 200
Response Headers: {'Content-Type': 'text/html; charset=utf-8', 'Content-Length': '8831', 'Connection': 'keep-alive', 'Cache-Control': 'no-cache, no-store, must-revalidate', 'Pragma': 'no-cache', 'Expires': '-1', 'Server': 'Microsoft-IIS/10.0', 'X-AspNet-Version': '4.0.30319', 'Set-Cookie': '__AntiXsrfToken=61e23491f8f54ab1b7b13183bae6d95b; path=/; secure; HttpOnly', 'X-XSS-Protection': '1; mode=block', 'X-UA-Compatible': 'IE=edge,chrome=1', 'Strict-Transport-Security': 'max-age=31536000; includeSubDomains', 'X-Content-Type-Options': 'nosniff', 'Date': 'Wed, 03 Dec 2025 23:47:01 GMT'}

Response Content Length: 8831 bytes
Response Text Length: 8831 characters

✅ Request successful!

--- First 1000 characters of response ---


<!DOCTYPE html>

<html lang="en">
<head><meta charset="utf-8" /><meta name="viewport" content="width=device-width, initial-scale=1.0" /><meta http-equiv="Content-Security-Policy" content="default-src &#39;self&#39;; script-src &#39;self&#39; &#39;un

## Parse Form Data (Optional)

If needed, we can parse the form data to see what fields are being sent.


In [ ]:
# Show what we're sending
from urllib.parse import parse_qs

parsed_body = parse_qs(body)

print("Form fields being sent:")
print("=" * 60)
for key, value in parsed_body.items():
    print(f"{key}: {value[0] if value else ''}")

print("\n" + "=" * 60)
print("Summary: Only sending borough, block, and lot - no VIEWSTATE or other fields")
print("=" * 60)


Form fields in the request:
__VIEWSTATE: [Very long encoded string - 12440 chars]
__VIEWSTATEGENERATOR: [Very long encoded string - 8 chars]
__SCROLLPOSITIONX: 0
__SCROLLPOSITIONY: 18.5
__EVENTVALIDATION: [Very long encoded string - 3371 chars]
ctl00$MainContent$ddlLeadAgency: XYU@2!
ctl00$MainContent$ddlCommunityDistrict: XYU@2!
ctl00$MainContent$ddlBorough: Brooklyn
ctl00$MainContent$txtBlock: 7061
ctl00$MainContent$txtLot: 27
ctl00$MainContent$btnSearch:  Search
ctl00$MainContent$grdSearchResults$ctl02$hfFiscalYr: 2008
ctl00$MainContent$grdSearchResults$ctl02$hfCEQRNum: 08DME007K
ctl00$MainContent$grdSearchResults$ctl02$hfMilestoneFolderName: tech_memo
ctl00$MainContent$grdSearchResults$ctl03$hfFiscalYr: 2024
ctl00$MainContent$grdSearchResults$ctl03$hfCEQRNum: 24HPD040K
ctl00$MainContent$grdSearchResults$ctl03$hfMilestoneFolderName: det_significance

Key Search Parameters:
Borough: Brooklyn
Block: 7061
Lot: 27


## Alternative: Get VIEWSTATE First

If the minimal request doesn't work, we might need to first GET the page to obtain a valid VIEWSTATE, then use it in the POST request.


In [ ]:
# Try getting the initial page first to extract VIEWSTATE
try:
    print("Fetching initial page to get VIEWSTATE...")
    initial_response = requests.get(url, timeout=30)
    
    if initial_response.status_code == 200:
        print("✅ Got initial page")
        
        # Extract VIEWSTATE using regex
        import re
        viewstate_match = re.search(r'name="__VIEWSTATE" id="__VIEWSTATE" value="([^"]+)"', initial_response.text)
        viewstate_gen_match = re.search(r'name="__VIEWSTATEGENERATOR" id="__VIEWSTATEGENERATOR" value="([^"]+)"', initial_response.text)
        eventval_match = re.search(r'name="__EVENTVALIDATION" id="__EVENTVALIDATION" value="([^"]+)"', initial_response.text)
        
        if viewstate_match:
            viewstate = viewstate_match.group(1)
            print(f"✅ Found VIEWSTATE ({len(viewstate)} chars)")
        else:
            print("⚠️  Could not find VIEWSTATE")
            viewstate = ""
            
        if viewstate_gen_match:
            viewstate_gen = viewstate_gen_match.group(1)
            print(f"✅ Found VIEWSTATEGENERATOR")
        else:
            viewstate_gen = ""
            
        if eventval_match:
            eventval = eventval_match.group(1)
            print(f"✅ Found EVENTVALIDATION ({len(eventval)} chars)")
        else:
            eventval = ""
        
        # Now make POST request with VIEWSTATE
        if viewstate:
            print("\nMaking POST request with VIEWSTATE...")
            form_data_with_viewstate = {
                "__VIEWSTATE": viewstate,
                "__VIEWSTATEGENERATOR": viewstate_gen if viewstate_gen else "F2CE38DF",
                "__EVENTVALIDATION": eventval if eventval else "",
                "ctl00$MainContent$ddlBorough": "Brooklyn",
                "ctl00$MainContent$txtBlock": "7061",
                "ctl00$MainContent$txtLot": "27",
                "ctl00$MainContent$btnSearch": "Search"
            }
            
            body_with_viewstate = urlencode(form_data_with_viewstate)
            
            post_response = requests.post(url, headers=headers, data=body_with_viewstate, timeout=30)
            
            print(f"\nStatus Code: {post_response.status_code}")
            print(f"Response Length: {len(post_response.text)} chars")
            
            if 'grdSearchResults' in post_response.text or 'Search Results' in post_response.text:
                print("✅ Found search results!")
                # Extract CEQR numbers
                ceqr_pattern = r'\d{2}[A-Z]{3}\d{3}[A-Z]'
                ceqr_matches = re.findall(ceqr_pattern, post_response.text)
                if ceqr_matches:
                    print(f"✅ Found CEQR numbers: {set(ceqr_matches)}")
            elif 'Error' in post_response.text:
                print("⚠️  Got error page")
            else:
                print("📄 Response received")
                print(f"\nFirst 500 chars:\n{post_response.text[:500]}")
    else:
        print(f"❌ Failed to get initial page: {initial_response.status_code}")
        
except Exception as e:
    print(f"❌ Error: {e}")
